# 1. Import necessary library

In [ ]:
# Turn warnings off to keep notebook tidy
import warnings
warnings.filterwarnings("ignore")
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.inspection import permutation_importance

# 2. Loading data

In [2]:
UV_data_location = "data/UV_unmixing_result/Dataset_5refs_11 metals_0.5_PBA_composition.csv"
original_df = pd.read_csv(UV_data_location, header=0, index_col=0)
original_df.head(5)

,Al,Sc,V,Cr,Mn,Fe(II),Fe(III),Co,Cu,Zn,In,MA-Fe,MA-Co,c_2py,c_epox,c_uf1,c_uf3,c_uf4
0,0,0,0,0,0,0,0,0,0,0,100,100,0,0.297144,0.000000,0.000000,0.067720,0.000000
1,0,0,0,0,0,0,0,0,0,50,50,100,0,0.308718,0.000000,0.000000,0.077348,0.000000
2,0,0,0,0,0,0,0,0,0,100,0,100,0,0.309060,0.000000,0.000000,0.058028,0.000000
3,0,0,0,0,0,0,0,0,50,0,50,100,0,0.280478,0.000000,0.000000,0.092470,0.000000
4,0,0,0,0,0,0,0,0,50,50,0,100,0,0.017312,0.549544,0.014336,0.000000,0.050738


## Incorporating average metal valence for B site metals

In [3]:
Bmetal_list = ['Al', 'Sc', 'V', 'Cr', 'Mn', 'Fe(II)', 'Fe(III)', 'Co', 'Cu', 'Zn', 'In']

metal_valence = [3, 3, 3, 3, 2, 2, 3, 2, 2, 2, 3]
original_df['avg_valence'] = 0

# atomic weight can also be incorporated if needed
# metal_MW = [26.98, 44.96, 50.94, 51.99, 54.94, 55.85, 55.85, 58.93, 63.55, 65.38, 114.82]
# original_df['avg_MW'] = 0

for i, metal in enumerate(Bmetal_list):
    original_df['avg_valence'] += original_df[metal] * metal_valence[i] /100
    #original_df['avg_MW'] += original_df[metal] * metal_MW[i] /100

original_df

,Al,Sc,V,Cr,Mn,Fe(II),Fe(III),Co,Cu,Zn,In,MA-Fe,MA-Co,c_2py,c_epox,c_uf1,c_uf3,c_uf4,avg_valence
0,0,0,0,0,0,0,0,0,0,0,100,100,0,0.297144,0.000000,0.000000,0.067720,0.000000,3.0
1,0,0,0,0,0,0,0,0,0,50,50,100,0,0.308718,0.000000,0.000000,0.077348,0.000000,2.5
2,0,0,0,0,0,0,0,0,0,100,0,100,0,0.309060,0.000000,0.000000,0.058028,0.000000,2.0
3,0,0,0,0,0,0,0,0,50,0,50,100,0,0.280478,0.000000,0.000000,0.092470,0.000000,2.5
4,0,0,0,0,0,0,0,0,50,50,0,100,0,0.017312,0.549544,0.014336,0.000000,0.050738,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,50,0,0,0,50,0,0,0,0,0,0,0,100,0.189284,0.034487,0.000000,0.219271,0.000000,2.5
194,50,0,0,50,0,0,0,0,0,0,0,0,100,0.379010,0.000000,0.000000,0.053758,0.000000,3.0
195,50,0,50,0,0,0,0,0,0,0,0,0,100,0.348805,0.000000,0.000000,0.064849,0.000000,3.0
196,50,50,0,0,0,0,0,0,0,0,0,0,100,0.422008,0.000000,0.000000,0.036115,0.000000,3.0


# 3. Spliting data for 5CV

In [ ]:
##################################################################################
### Note: This part has been done already, you can uncomment and run if needed.###
##################################################################################


# # settings
# target_col = None          # e.g., "label" or leave as None for plain KFold
# n_splits = 5
# shuffle = True
# random_state = 42

# df = original_df.copy()

# rng = np.random.RandomState(random_state) if shuffle else None
# idx = np.arange(len(df))
# if shuffle:
#     rng.shuffle(idx)

# if target_col:
#     # Stratified: split by class proportions
#     df["_tmp_idx"] = np.arange(len(df))
#     fold_col = np.full(len(df), -1, dtype=int)
#     for cls, sub in df.groupby(target_col):
#         sub_idx = sub.index.to_numpy()
#         # shuffle within class
#         if shuffle:
#             rng.shuffle(sub_idx)
#         # chunk into folds
#         parts = np.array_split(sub_idx, n_splits)
#         for fold_id, part in enumerate(parts):
#             fold_col[part] = fold_id
#     df["fold"] = fold_col
#     df.drop(columns=["_tmp_idx"], inplace=True)
# else:
#     # Plain KFold by index
#     parts = np.array_split(idx, n_splits)
#     fold_col = np.full(len(df), -1, dtype=int)
#     for fold_id, part in enumerate(parts):
#         fold_col[part] = fold_id
#     df["fold"] = fold_col

# # Export five CSVs (train/test per fold)
# for k in range(n_splits):
#     test = df[df["fold"] == k].drop(columns=["fold"])
#     train = df[df["fold"] != k].drop(columns=["fold"])
#     test.to_csv(f"data/XGB_dataset/fold{k+1}_test.csv", index=False)
#     train.to_csv(f"data/XGB_dataset/fold{k+1}_train.csv", index=False)

# 4. XGBModel training

In [ ]:
# Load the data splits for 5-fold CV
train_data, test_data = [], []
data_loc = 'data/XGB_dataset/'

for i in range(5):
    
    train_data.append(pd.read_csv(data_loc + f'fold{i+1}_train.csv'))
    test_data.append(pd.read_csv(data_loc + f'fold{i+1}_test.csv'))


# extra features like 'MA-Co' and 'avg_valence' can be added to the list below for model training if needed

available_features = ['Al','Sc','V','Cr','Mn','Fe(II)','Fe(III)','Co','Cu','Zn','In','MA-Fe']
# available_features.append('MA-Co')
# available_features.append('avg_valence')

## 4.1 Forward feature selection by R2

In [5]:
r2_by_feature_number = []
r2_by_feature_number_kfold = []
chosen_features = []
r2_features = available_features.copy()

number_of_features = len(r2_features)

for i in range(number_of_features):
    best_result = -np.inf
    best_feature = None
    best_result_kfold = None

    for feature in r2_features:
        features_to_use = chosen_features + [feature]
        feature_r2_kfold = []

        for k_fold in range(5):
            train = train_data[k_fold]
            test = test_data[k_fold]

            X_train = train[features_to_use]
            X_test  = test[features_to_use]
            y_train = train['c_2py']
            y_test  = test['c_2py']

            model = XGBRegressor(
                random_state=42,
                learning_rate=0.1,
                n_estimators=200,
                max_depth=4,
                subsample=0.8,
                colsample_bytree=0.8,
                n_jobs=-1
            )
            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            feature_r2_kfold.append(r2_score(y_test, y_pred))

        feature_r2_mean = float(np.mean(feature_r2_kfold))

        if feature_r2_mean > best_result:
            best_result = feature_r2_mean
            best_result_kfold = feature_r2_kfold
            best_feature = feature

    r2_by_feature_number.append(best_result)
    r2_by_feature_number_kfold.append(best_result_kfold)
    chosen_features.append(best_feature)
    r2_features.remove(best_feature)

    print(f'Feature {i+1:2d}: {best_feature}, R²: {best_result:.3f}')

Feature  1: Cu, R²: 0.308
Feature  2: MA-Fe, R²: 0.352
Feature  3: Co, R²: 0.482
Feature  4: Mn, R²: 0.541
Feature  5: Zn, R²: 0.574
Feature  6: In, R²: 0.615
Feature  7: V, R²: 0.627
Feature  8: Al, R²: 0.626
Feature  9: Fe(II), R²: 0.611
Feature 10: Sc, R²: 0.595
Feature 11: Fe(III), R²: 0.552
Feature 12: Cr, R²: 0.556


## 4.2 Forward feature selection by MAE

In [7]:
MAE_by_feature_number = []
MAE_by_feature_number_kfold = []
chosen_features = []
mae_features = available_features.copy()

number_of_features = len(mae_features)

for i in range(number_of_features):
    best_score = np.inf
    best_feature = None
    best_scores_kfold = None

    for feature in mae_features:
        features_to_use = chosen_features + [feature]
        fold_scores = []

        for k_fold in range(5):
            train = train_data[k_fold]
            test  = test_data[k_fold]

            X_train = train[features_to_use]
            X_test  = test[features_to_use]
            y_train = train['c_2py']
            y_test  = test['c_2py']

            model = XGBRegressor(
                random_state=42,
                learning_rate=0.1,
                n_estimators=300,
                max_depth=4,
                subsample=0.8,
                colsample_bytree=0.8,
                n_jobs=-1
            )
            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            score = mean_absolute_error(y_test, y_pred)
            fold_scores.append(float(score))

        mean_score = np.mean(fold_scores)

        if mean_score < best_score:
            best_score = mean_score
            best_scores_kfold = fold_scores
            best_feature = feature

    MAE_by_feature_number.append(best_score)
    MAE_by_feature_number_kfold.append(best_scores_kfold)
    chosen_features.append(best_feature)
    mae_features.remove(best_feature)

    print(f"Feature {i+1:2d}: {best_feature}, MAE: {best_score:.5f}")

Feature  1: Cu, MAE: 0.03861
Feature  2: MA-Fe, MAE: 0.03551
Feature  3: Co, MAE: 0.03048
Feature  4: Zn, MAE: 0.02745
Feature  5: In, MAE: 0.02602
Feature  6: Mn, MAE: 0.02535
Feature  7: V, MAE: 0.02485
Feature  8: Al, MAE: 0.02448
Feature  9: Fe(II), MAE: 0.02403
Feature 10: Sc, MAE: 0.02396
Feature 11: Cr, MAE: 0.02566
Feature 12: Fe(III), MAE: 0.02611


## 4.3 Permutation importance

In [9]:
def permutation_importance_one_split(train, test, features, target="c_2py",
                                     metric="r2", n_repeats=10, random_state=42):
    X_train, y_train = train[features], train[target]
    X_test,  y_test  = test[features],  test[target]

    model = XGBRegressor(
        random_state=random_state, learning_rate=0.1, n_estimators=300,
        max_depth=4, subsample=0.8, colsample_bytree=0.8, n_jobs=-1, tree_method="hist"
    )
    model.fit(X_train, y_train)

    pi = permutation_importance(
        model, X_test, y_test,
        scoring= metric,
        n_repeats=n_repeats, random_state=random_state, n_jobs=-1
    )

    df = pd.DataFrame({
        "feature": features,
        "importance_mean": pi.importances_mean,
        "importance_std":  pi.importances_std
    }).sort_values("importance_mean", ascending=False, ignore_index=True)

    # For RMSE/MAE (neg scorers), positive "score_drop" is easier to read:
    if metric in {"rmse", "mae"}:
        df["score_drop"] = -df["importance_mean"]   # larger drop => more important

    return df, model

def permutation_importance_cv(train_folds, test_folds, features, target="c_2py",
                              metric="r2", n_repeats=10, random_state=42):
    per_fold = []
    for k in range(len(train_folds)):
        df_k, _ = permutation_importance_one_split(
            train_folds[k], test_folds[k], features, target, metric, n_repeats, random_state
        )
        df_k = df_k.set_index("feature")
        per_fold.append(df_k["importance_mean"])

    M = pd.concat(per_fold, axis=1)
    M.columns = [f"fold_{i}" for i in range(len(train_folds))]
    out = pd.DataFrame({
        "feature": M.index,
        "importance_mean_cv": M.mean(axis=1).values,
        "importance_std_cv":  M.std(axis=1, ddof=1).values
    }).sort_values("importance_mean_cv", ascending=False, ignore_index=True)

    if metric in {"rmse", "mae"}:
        out["score_drop_cv"] = -out["importance_mean_cv"]

    return out, M  # M has per-fold values if you want to inspect stability

In [10]:
pi_features = available_features.copy()
pi_cv, per_fold = permutation_importance_cv(train_data, test_data, pi_features, metric="r2", n_repeats=20)
pi_cv

,feature,importance_mean_cv,importance_std_cv
0,Cu,1.028299,0.478631
1,MA-Fe,0.567744,0.267497
2,Zn,0.209222,0.120843
3,Co,0.169598,0.166314
4,Mn,0.127270,0.046032
5,In,0.062803,0.036740
6,Sc,0.018448,0.051610
7,V,0.016571,0.012195
8,Fe(III),0.003880,0.022357
9,Cr,0.003753,0.008157
